<a href="https://colab.research.google.com/github/FRC4188/Edge_Computing/blob/main/tf1_edgetpu_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Make Directories

In [ ]:
MODEL_DIR = 'model/'
PRETRAINED_DIR = MODEL_DIR + 'pretrained/'
CUSTOM_DIR = MODEL_DIR + 'custom/'
EXPORT_DIR = CUSTOM_DIR + 'export/'
DATASET_DIR = 'dataset/'
COMPILED_DATASET_DIR = MODEL_DIR + 'dataset/'

%mkdir {MODEL_DIR} 
%mkdir {PRETRAINED_DIR}
%mkdir {CUSTOM_DIR}
%mkdir {EXPORT_DIR}
%mkdir {DATASET_DIR}
%mkdir {COMPILED_DATASET_DIR}

Switch to Tensorflow 1.x (deprecated August 1, 2022 at time of writing). Use one or the other, not both

In [ ]:
%tensorflow_version 1.x

In [ ]:
!pip install --upgrade tensorflow-gpu==1.15.2

Install Libraries

In [ ]:
!git clone --quiet https://github.com/tensorflow/models.git

!pip install tf_slim

!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib

!pip install -q pycocotools

!pip install lvis

!cd models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf1/setup.py . && pip install .

import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

!pip install numpy==1.19.5
!pip uninstall -y pycocotools
!pip install pycocotools --no-binary pycocotools

Test Setup (look for OK at end)

In [ ]:
!python models/research/object_detection/builders/model_builder_tf1_test.py

#Import Dataset

1. Collect pictures and/or videos of the object(s) you are trying to detect. Make sure you get it in various lightings and backgrounds so that your model can generalize the object when it is detecting it. 

2. Make sure the pictures are in `.jpg`, `.png`, `.bmp ` format and videos are in `.mp4`, `.mov`, `.avi` format. 

3. Navigate to [Roboflow](https://app.roboflow.com/) and create an account. Roboflow is an online annotator that allows us to configure our dataset for training. Follow all the steps to create an account. Then, make a new project and give it an appropriate name. Next, give the annotation group an appropriate name. For example, if I were trying to detect different brand of cars like Ford, Nissan or BMW, I would name the annotation group "car brands". 

4. Go inside your project and begin uploading the pictures and/or videos you took into the dataset by clicking the "Upload" tab. After it has loaded all of it in, click the green "Finish Uploading" button in the upper righthand corner. A pop-up will appear, prompting you to split the dataset into train, validation, and test sets. Change it if necessary. 

5. Begin annotating the dataset. Click onto an image and select the box tool. Draw a box around your object and only the object. Try to isolate it as much as possible, but do not worry about being perfect. Once the box has been drawn, make sure to give it an appropriate identifying class name. Once you've done so, you can reuse the class name when annotating other images of the same type. For a large dataset, annotating can be tedious. However, if you have labeled enough, you will be able to quickly train a dataset based on what you've annotated so far. It can then attempt to label your images for you and you just have to check through for any missed or incorrect labels. Also, if your object is general enough, you can use one that is built-in. 

6. Export your model version. Navigate out of the images by clicking the back arrow and then click "Dataset" on the sidebar. Generate a new version. You can add preprocessing steps in order to improve your training. Once you've chosen the desired steps, you can also choose augmentation steps. This will increase the generality of your model, i.e. how effective your model is in a variety of situations. Go to the next step and generate the model. When the version has finished generating, click on the version you just created and click "Export". On format, select "TFRecords". Also, select "show download code". Make sure to select "Terminal". Copy all the text in the box and paste it into the below form.

Download dataset from Roboflow

In [ ]:
DOWNLOAD_COMMAND = "curl -L \"https://app.roboflow.com/ds/Wd8kVxzsdN?key=Rj2UTtkvU9\" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip" #@param {type:"string"}
!cd dataset && {DOWNLOAD_COMMAND}

Move dataset files and rename them

In [ ]:
TRAIN_FILE = COMPILED_DATASET_DIR + 'test.tfrecord'
TEST_FILE = COMPILED_DATASET_DIR + 'train.tfrecord'
LABELS_FILE = COMPILED_DATASET_DIR + 'labels.pbtxt'

!cp dataset/test/*.tfrecord {TRAIN_FILE}
!cp dataset/train/*.tfrecord {TEST_FILE}
!cp dataset/train/*.pbtxt {LABELS_FILE}

# Download Pretrained Model

Download `ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03` model from Tensorflow 1 Model Zoo

In [ ]:
MODEL_LINK = 'http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz' #@param {type:"string"}

!cd {PRETRAINED_DIR} && wget {MODEL_LINK}

Extract model folder from file 

In [ ]:
!cd {PRETRAINED_DIR} && tar -zxvf *.tar.gz

# Configure pipeline file

Input number of training steps

In [ ]:
NUM_TRAIN_STEPS = 1500 #@param {type:"integer"}
NUM_EVAL_STEPS = int(NUM_TRAIN_STEPS / 15)

Collect Directories

In [ ]:
MODEL_NAME = MODEL_LINK[MODEL_LINK.index('_detection/') + 11 : MODEL_LINK.index('.tar.gz')]

CHECKPOINTS_DIR = PRETRAINED_DIR + MODEL_NAME + '/model.ckpt'
CONFIG_FILE = PRETRAINED_DIR + MODEL_NAME + '/pipeline.config' 
CUSTOM_CONFIG_FILE = CUSTOM_DIR + "pipeline.config"

Count number of classes from labels file

In [ ]:
def get_num_classes(pbtxt_fname):
  from object_detection.utils import label_map_util
  label_map = label_map_util.load_labelmap(pbtxt_fname)
  categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=90, use_display_name=True)
  category_index = label_map_util.create_category_index(categories)
  return len(category_index.keys())

Configure pipeline file

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

config = config_util.get_configs_from_pipeline_file(CONFIG_FILE)

pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_FILE, "r") as f:                                                                                                                                                                                                                     
  proto_str = f.read()                                                                                                                                                                                                                                          
  text_format.Merge(proto_str, pipeline_config)  

pipeline_config.model.ssd.num_classes = get_num_classes(LABELS_FILE)
pipeline_config.train_config.batch_size = 16
pipeline_config.train_config.num_steps = NUM_TRAIN_STEPS
pipeline_config.train_config.fine_tune_checkpoint = CHECKPOINTS_DIR
pipeline_config.train_input_reader.label_map_path = LABELS_FILE
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [TRAIN_FILE]
pipeline_config.eval_input_reader[0].label_map_path = LABELS_FILE
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [TEST_FILE]
pipeline_config.eval_config.include_metrics_per_category = False
pipeline_config.eval_config.num_examples = NUM_EVAL_STEPS

config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CUSTOM_CONFIG_FILE, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

View file

In [ ]:
!cat {CUSTOM_CONFIG_FILE}

# Train the model

In [ ]:
OVERRIDE_TRAIN_STEPS = 0#@param {type:"integer"}
OVERRIDE_EVAL_STEPS = OVERRIDE_TRAIN_STEPS / 15

!python /content/models/research/object_detection/model_main.py \
  --pipeline_config_path={CUSTOM_CONFIG_FILE} \
  --model_dir={CUSTOM_DIR} \
  --num_train_steps={OVERRIDE_TRAIN_STEPS} \
  --num_eval_steps={OVERRIDE_EVAL_STEPS} \
  --alsologtostderr


# View training and validation with Tensorboard

In [ ]:
%tensorboard --logdir {CUSTOM_DIR + 'events.out.*'}

# Export for TFLite

In [ ]:
!python models/research/object_detection/export_tflite_ssd_graph.py \
  --pipeline_config_path {pipeline_fname} \
  --trained_checkpoint_prefix {CUSTOM_DIR + 'model.ckpt-' + NUM_TRAIN_STEPS} \
  --output_directory {EXPORT_DIR} \
  --add_postprocessing_op

# Convert to TFLite

In [ ]:
INPUT_NAME = 'serving_default_input:0'
OUT0_NAME = 'StatefulPartitionedCall:3;StatefulPartitionedCall:2;StatefulPartitionedCall:1;StatefulPartitionedCall:0'
OUT1_NAME = 'StatefulPartitionedCall:3;StatefulPartitionedCall:2;StatefulPartitionedCall:1;StatefulPartitionedCall:01'
OUT2_NAME = 'StatefulPartitionedCall:3;StatefulPartitionedCall:2;StatefulPartitionedCall:1;StatefulPartitionedCall:02'
OUT3_NAME = 'StatefulPartitionedCall:3;StatefulPartitionedCall:2;StatefulPartitionedCall:1;StatefulPartitionedCall:03'
OUTPUT_NAME = OUT0_NAME + ',' + OUT1_NAME + ',' + OUT2_NAME + ',' + OUT3_NAME

!tflite_convert \
  --output_file {EXPORT_DIR + 'tflite/converted.tflite'} \
  --graph_def_file {EXPORT_DIR + 'tflite/tflite_graph.pb'} \
  --inference_type QUANTIZED_UINT8 \
  --input_arrays {INPUT_NAME} \
  --output_arrays {OUTPUT_NAME}\
  --mean_values=128 \
  --std_dev_values=128 \
  --input_shapes=1,300,300,3 \
  --change_concat_input_ranges False \
  --allow_nudging_weights_to_use_fast_gemm_kernel True \
  --allow_custom_ops

# Convert to EdgeTPU

In [ ]:
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
!sudo apt-get update
!sudo apt-get install edgetpu-compiler	

In [ ]:
!edgetpu_compiler {EXPORT_DIR + 'tflite/converted.tflite'}